In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import re
import datetime as dt
import os
pd.set_option('display.max_columns', None) 

In [2]:
url = 'https://milwaukee.craigslist.org/search/eby/apa?bedrooms=1&pets_dogs=1'
api_key = '843d4869e634a9bb3af33c77fe1dab4a'
api_pw = '24065c07e5534ee6fb4ceba8ccdea42e'

In [3]:
open_site = requests.get(url)

In [4]:
parse_text_1 = bs(open_site.text, 'html.parser').prettify()
parse_text_2 = bs(open_site.text, 'html.parser')
print(parse_text_1)

﻿
<!DOCTYPE html>
<html class="no-js">
 <head>
  <title>
   milwaukee apts/housing for rent  - craigslist
  </title>
  <meta content="milwaukee apts/housing for rent  - craigslist" name="description">
   <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
   <link href="https://milwaukee.craigslist.org/search/eby/apa" rel="canonical">
    <link href="https://milwaukee.craigslist.org/search/eby/apa?format=rss&amp;min_bedrooms=1" rel="alternate" title="RSS feed for craigslist | milwaukee apts/housing for rent  - craigslist" type="application/rss+xml">
     <meta content="width=device-width,initial-scale=1" name="viewport">
      <link href="//www.craigslist.org/styles/cl.css?v=281764e2707bd58e05233e4b7df36df8" media="all" rel="stylesheet" type="text/css">
       <link href="//www.craigslist.org/styles/search.css?v=2d972697128e23b83898e73d71ad0d39" media="all" rel="stylesheet" type="text/css">
        <link href="//www.craigslist.org/styles/jquery-ui-clcustom.css?v=3b05ddffb7c7f5b62066

In [5]:
listings = parse_text_2.find_all("p","result-info")
listings[0]

<p class="result-info">
<span class="icon icon-star" role="button">
<span class="screen-reader-text">favorite this post</span>
</span>
<time class="result-date" datetime="2019-05-05 14:43" title="Sun 05 May 02:43:17 PM">May  5</time>
<a class="result-title hdrlnk" data-id="6871237633" href="https://milwaukee.craigslist.org/apa/d/milwaukee-3-br-15-bath-close-to-medical/6871237633.html">3 BR 1.5 Bath- Close to Medical College of WI</a>
<span class="result-meta">
<span class="result-price">$1795</span>
<span class="housing">
                    3br -
                    2186ft<sup>2</sup> -
                </span>
<span class="result-hood"> (425 N. 61st st)</span>
<span class="result-tags">
                    pic
                    <span class="maptag" data-pid="6871237633">map</span>
</span>
<span class="banish icon icon-trash" role="button">
<span class="screen-reader-text">hide this posting</span>
</span>
<span aria-hidden="true" class="unbanish icon icon-trash red" role="button"></s

In [6]:
n=1
df = pd.DataFrame()
for i in listings:
    index='listing #'+str(n)
    try: 
        description = i.find('a','result-title').contents
    except:
        description = 'Not Available'
    try:
        price = i.find('span','result-price').contents
    except:
        price = 'Not Available'
    try:
        location = i.find('span','result-hood').contents
    except:
        location = 'Not Available'
    try:
        details = i.find('span','housing').contents[0].replace(' ','').replace('\n','').replace('-','')
        details_split = details.split('br')
        br = details_split[0]
        try:
            sq_ft = details_split[1]
        except:
            sq_ft = 'Not Available'
    except:
        details = 'Not Available'
    try:
        anchor_link = i.find('a','result-title hdrlnk',href=True)
        link = anchor_link['href']
    except:
        link='No Link Available'
    full_list = [location, price, br, sq_ft,description,link]
    full_list = pd.Series(full_list)
    df = df.append(full_list, ignore_index=True)
    n=n+1

In [7]:
df = df.rename(columns={0:'location',1:'rent $',2:'bedrooms',3:'sq_ft',4:'description',5:'link'})
df = df.astype(str)
df['location'] = df['location'].str.replace('[','').str.replace(']','').str.replace('(','').str.replace(')','').str.replace("'",'').str.strip().str.capitalize()
df['rent $'] = df['rent $'].str.replace('[','').str.replace(']','').str.replace('$','').str.replace("'",'').str.strip()
df['rent $'] = df['rent $'].astype(int)
df['bedrooms'] = df['bedrooms'].astype(int)
df['sq_ft'] = df['sq_ft'].str.replace('ft','').str.strip()
df['description'] = df['description'].str.replace('[','').str.replace(']','').str.replace('$','').str.replace("'",'').str.strip()

In [8]:
df = df.sort_values(by='rent $', ascending=True)
df_filtered = df[df['location']!='Not available'].reset_index(drop=True)

In [9]:
df_filtered

,location,rent $,bedrooms,sq_ft,description,link
0,1716-22 north 5th street,565,1,470,Near Fiserv Arena and downtown! Redecorated 1 BRs,https://milwaukee.craigslist.org/apa/d/milwauk...
1,"West allis, wi 53214",650,2,,Remodeled 2bedroom upper,https://milwaukee.craigslist.org/apa/d/milwauk...
2,2774 s. 60th st. milwaukee,725,2,,2 bedroom lower - Large 5 room Apt. 725 small ...,https://milwaukee.craigslist.org/apa/d/2-bedro...
3,2774 s. 60th st. milwaukee,725,2,,2 bedroom lower - Large 5 room Apt. 725 small ...,https://milwaukee.craigslist.org/apa/d/2-bedro...
4,2762 s. 60th st. milwaukee,725,2,,2 bedroom upper - Large 5 room Apt. 725 small ...,https://milwaukee.craigslist.org/apa/d/2-bedro...
5,2762 s. 60th st. milwaukee,725,2,,2 bedroom upper - Large 5 room Apt. 725 small ...,https://milwaukee.craigslist.org/apa/d/2-bedro...
6,2556 n. murray,765,1,635,1 bed 1 bath apartment Ideal Eastside location,https://milwaukee.craigslist.org/apa/d/milwauk...
7,603a e. burleigh st. - riverwest,775,2,750,!OPEN HOUSE! Large 2 Bedroom Upper at Great Ri...,https://milwaukee.craigslist.org/apa/d/milwauk...
8,1160 e. singer cr.,795,2,900,"!OPEN HOUSE! 2 Br Apt w/ HEAT, WATER, PARKING ...",https://milwaukee.craigslist.org/apa/d/milwauk...
9,2428g n. bremen st. - riverwest,825,2,750,!OPEN HOUSE! Remodeled 2 Br Polish Flat at Gre...,https://milwaukee.craigslist.org/apa/d/milwauk...


In [10]:
date = dt.datetime.now()
month = date.month
month = str(month)
day = date.day
day = str(day)
year = date.year
year = str(year)
hour = date.hour
hour = str(hour)
minute = date.minute
minute = str(minute)
pwd = os.getcwd()
xl_title = pwd+'/Apt List as of '+month+'-'+day+'-'+year+' '+hour+':'+minute+'.xlsx'
xl_title = str(xl_title)

In [11]:
df_filtered.to_excel(xl_title, engine='xlsxwriter')